In [ ]:
!pip install openai chromadb python-dotenv gTTS SpeechRecognition python-docx

In [ ]:
!pip install openai==0.28

In [ ]:
import os
import openai
from datetime import datetime
import chromadb
from gtts import gTTS
from IPython.display import Audio, display
import speech_recognition as sr
from google.colab import files

In [ ]:
class Config:
    def __init__(self):
        self.chroma_client = chromadb.Client()
        # Get or create the collection instead of always creating it
        self.collection = self.chroma_client.get_or_create_collection("training_sessions")
        self.recognizer = sr.Recognizer()

    def save_to_chroma(self, module, user_input, feedback):
        self.collection.add(
            documents=[user_input],
            metadatas=[{
                "module": module,
                "feedback": feedback,
                "timestamp": datetime.now().isoformat()
            }],
            ids=[str(datetime.now().timestamp())]
        )


In [ ]:
class AITrainer:
    def __init__(self):
        from dotenv import load_dotenv
        load_dotenv()
        openai.api_key = os.getenv("OPENAI_API_KEY")

    def get_feedback(self, prompt, context):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": f"You are a communication coach. Provide feedback on: {context}"
            }, {
                "role": "user",
                "content": prompt
            }]
        )
        return response.choices[0].message.content


In [ ]:
class VoiceHandler:
    def __init__(self):
        self.config = Config()

    def upload_audio(self):
        uploaded = files.upload()
        for fn in uploaded.keys():
            # Check if the file is in a supported format before proceeding
            if not fn.lower().endswith(('.wav', '.aiff', '.aifc', '.flac')):
                print(f"Error: Unsupported file format. Please upload a WAV, AIFF, AIFF-C, or FLAC file.")
                return None # or handle the error as you prefer

            with sr.AudioFile(fn) as source:
                audio = self.config.recognizer.record(source)
                try:
                    return self.config.recognizer.recognize_google(audio)
                except sr.UnknownValueError:
                    return "Could not understand audio"

    def text_to_speech(self, text):
        tts = gTTS(text=text, lang='en')
        tts.save("response.mp3")
        display(Audio("response.mp3", autoplay=True))

In [ ]:
class TrainingModules:
    def __init__(self):
        self.ai = AITrainer()
        self.voice = VoiceHandler()
        self.config = Config()

    def run_impromptu(self):
        topic = "Explain why effective communication matters"
        print(f"Speak about: {topic}")

        # Upload audio file in Colab
        print("Please upload your speech recording:")
        transcript = self.voice.upload_audio()

        feedback = self.ai.get_feedback(
            f"Impromptu speech transcript: {transcript}",
            "structure, clarity, and engagement"
        )
        self.config.save_to_chroma("impromptu", transcript, feedback)
        print("\nFeedback:")
        self.voice.text_to_speech(feedback)
        return feedback

    def run_chat_mode(self):
        print("Chat mode - type your message (or 'exit' to quit):")
        while True:
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                break

            feedback = self.ai.get_feedback(
                user_input,
                "tone, clarity, and conversational effectiveness"
            )
            self.config.save_to_chroma("chat", user_input, feedback)
            print("\nCoach:")
            self.voice.text_to_speech(feedback)


In [ ]:
def main():
    trainer = TrainingModules()

    print("1. Impromptu Speaking\n2. Chat Practice")
    choice = input("Choose an option (1/2): ")

    if choice == '1':
        trainer.run_impromptu()
    elif choice == '2':
        trainer.run_chat_mode()
    else:
        print("Invalid choice")

if __name__ == "__main__":
    # First time setup - upload your .env file with OPENAI_API_KEY
    print("Please upload your .env file:")
    files.upload()
    main()


Please upload your .env file:
